In [1]:

import torch
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np
label_path = 'data/article/網路危機_A1_文章危機程度.xlsx'
# label_path = 'data/article/partial_A1_article.xlsx'
df = pd.read_excel(label_path)
df = df[['TextID','Title','Crisis_Level']]
print(df.columns)
# print(df)

for idx, val in enumerate(df['TextID']):
    if type(val) != str:
        df['TextID'][idx] = str(val)
stat = [0]*4
for idx, val in enumerate(df['Crisis_Level']):
    stat[val] += 1
print(stat)
four_label = df.values
title_list = df['Title'].values
textID_list = df['TextID'].values
sentence_path = 'data/raw/new_clean_data_all.xlsx'
s_df = pd.read_excel(sentence_path)[[ 'TextID','Title', 'Sentence', '無標註', '自殺與憂鬱',
       '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為']]
for idx, val in enumerate(s_df['TextID']):
    if type(val) != str:
        s_df['TextID'][idx] = str(val)

sentence_label_list = ['無標註', '自殺與憂鬱', '無助或無望', '正向文字', '其他負向文字',
       '生理反應或醫療狀況', '自殺行為']
other_categories = ['無助或無望', '正向文字', '其他負向文字','生理反應或醫療狀況']
article_dict = {str(key): {'無標註':'', '自殺與憂鬱':'',
       '無助或無望':'', '正向文字':'', '其他負向文字':'', '生理反應或醫療狀況':'', '自殺行為':'', '其他類型':''} for key in textID_list}
problem_title = []
for index, row in s_df.iterrows():
    article_dict[str(row['TextID'])]['label'] = df[(df['TextID']) == (row['TextID'])]['Crisis_Level']
    flag = article_dict[str(row['TextID'])]['label'].values[0]
    if flag == 3:
       flag = 1
    elif flag == 1:
       flag = 3
    article_dict[str(row['TextID'])]['Crisis_Level'] = flag
    article_dict[str(row['TextID'])]['label'] = [0.0, 0.0, 0.0, 0.0]
    article_dict[str(row['TextID'])]['label'][flag] += 1.0
    for s_label in sentence_label_list:
      if row[s_label] == 1:
        tar = ' ' + str(row['Sentence'])
        article_dict[str(row['TextID'])][s_label] += tar
        if s_label in other_categories:
            article_dict[str(row['TextID'])]['其他類型'] += tar
        elif s_label == '正向文字':
            article_dict[str(row['TextID'])]['無標註'] += tar
        break
all_article = []
for dic in article_dict:
    single_article = pd.DataFrame([article_dict[dic]])
    single_article['TextID'] = dic
    all_article.append(single_article)
all_article_df = pd.concat(all_article,axis=0, ignore_index=True)
print(all_article_df.columns)
all_article_df['TextID'] = all_article_df['TextID'].astype(str)
# all_article_df.to_excel('data/article/all_article_with_sentences_split_v2.xlsx', index=False)
all_article_df.to_excel('data/article/all_article_with_sentences_split_1108.xlsx', index=False)

Index(['TextID', 'Title', 'Crisis_Level'], dtype='object')
[672, 95, 200, 457]


/tmp/ipykernel_44257/4201800360.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TextID'][idx] = str(val)
/tmp/ipykernel_44257/4201800360.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_df['TextID'][idx] = str(val)


Index(['無標註', '自殺與憂鬱', '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為', '其他類型',
       'label', 'Crisis_Level', 'TextID'],
      dtype='object')


In [2]:
import torch
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np
all_df = pd.read_excel('data/article/all_article_with_sentences_split_v2.xlsx')
# sentence_label_list = ['無標註', '自殺與憂鬱', '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為']
sentence_label_list = ['無標註','自殺與憂鬱','其他負向文字', '自殺行為']
seed = 1
test_df = all_df.sample(frac=0.1, random_state=seed)
print(test_df.index)
without_test_df = all_df.drop(test_df.index)
# for index, row in all_df.iterrows():


Int64Index([ 325,  283, 1140,  236,  938,  571,   80, 1057,  953, 1404,
            ...
             321,  301, 1272,  950, 1041, 1177, 1232,  941, 1040,   94],
           dtype='int64', length=142)


In [14]:
statss = [0]*4
for index, row in test_df.iterrows():
    statss[row['Crisis_Level']] += 1
print(statss)
test_df.to_excel('data/article/test_article_1108.xlsx', index=False)

[72, 38, 23, 9]


In [4]:
statsss = [0]*4
for index, row in without_test_df.iterrows():
    statsss[row['Crisis_Level']] += 1
print(statsss)

[600, 419, 177, 86]


In [5]:
df_crisis_level_0 = without_test_df.loc[without_test_df['Crisis_Level'] == 0]
df_augmentation_0 = df_crisis_level_0.sample(n=100, random_state=seed)
df_remaining = without_test_df.drop(df_augmentation_0.index) 

In [6]:
statssss = [0]*4
for index, row in df_remaining.iterrows():
    statssss[row['Crisis_Level']] += 1
print(statssss)

[500, 419, 177, 86]


In [7]:
df_augmentation_0.to_excel('data/article/augmentation_article_0_1108.xlsx', index=False)

In [8]:
df_crisis_level_1 = df_remaining.loc[without_test_df['Crisis_Level'] == 1]
df_augmentation_1 = df_crisis_level_1.sample(n=100, random_state=1)
df_remaining_v2 = df_remaining.drop(df_augmentation_1.index) 

In [9]:
# the_stat = [0]*8
# k = 0
# for index, row in df_augmentation_1.iterrows():
#     k += 1
#     for idx, key in enumerate(df_augmentation_1):
#         if str(row[key]) == 'nan':
#             the_stat[idx] += 1
# print(the_stat)
# statssss = [0]*4
# for index, row in df_remaining_v2.iterrows():
#     statssss[row['Crisis_Level']] += 1
# print(statssss)

In [10]:
df_augmentation_1.to_excel('data/article/augmentation_article_1_1108.xlsx', index=False)

In [11]:
df_remaining_v2.to_excel('data/article/train_article_1108.xlsx', index=False)

In [12]:
df_crisis_level_3 = df_remaining.loc[without_test_df['Crisis_Level'] == 3]

the_stat = [0]*8
k = 0
for index, row in df_crisis_level_3.iterrows():
    k += 1
    for idx, key in enumerate(df_augmentation_0):
        if str(row[key]) == 'nan':
            the_stat[idx] += 1
print(the_stat)

[2, 3, 69, 59, 7, 55, 24, 2]


In [1]:
import pandas as pd
A1_df = pd.read_excel('data/article/train_article_1108.xlsx')
A2_df = pd.read_excel('data/article/A2_final_wo_test_50_1113_v2.xlsx')
print(A1_df.columns)
print(A2_df.columns)
# retrieve the A2 columns of A1
A1_df = A1_df[A2_df.columns]
# concat A1 and A2 and save
A1_A2_df = pd.concat([A1_df, A2_df], axis=0, ignore_index=True)
# statistic of A1_A2_df Crisis_Level
statsssss = [0]*4
for index, row in A1_A2_df.iterrows():
    statsssss[row['Crisis_Level']] += 1
print(statsssss)

A1_A2_df.to_excel('data/article/train_A1_A2_1113_v2.xlsx', index=False)

Index(['無標註', '自殺與憂鬱', '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為', '其他類型',
       'label', 'Crisis_Level', 'TextID'],
      dtype='object')
Index(['無標註', '自殺與憂鬱', '自殺行為', '其他類型', 'label', 'Crisis_Level', 'TextID'], dtype='object')
[1229, 623, 287, 133]


In [13]:
# 產生部分文章
# import pandas as pd
# label_path = 'data/article/網路危機_A1_文章危機程度.xlsx'
# df = pd.read_excel(label_path)
# df = df[['TextID','Title','Crisis_Level']]
# print(df.columns)
# # print(df)

# for idx, val in enumerate(df['TextID']):
#     if type(val) != str:
#         df['TextID'][idx] = str(val)
# new_column = [0, 1, 2, 3]
# all_article_df = []
# for label in new_column:
#     all_article_df.append(df[df['Crisis_Level'] == label].loc[:])
# for i in all_article_df:
#     print(len(i))

# partial_article_df = []
# for idx, a_df in enumerate(all_article_df):
#     if idx == 1:
#         n0 = 95
#     else:
#         n0 = 200
#     partial_article_df.append(a_df.sample(n=n0, random_state=12345))
# for i in partial_article_df:
#     print(len(i))

# train_df = pd.concat(partial_article_df,axis=0, ignore_index=True)
# print(train_df.columns)
# train_df.to_excel('data/article/partial_A1_article.xlsx')
